In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.model_selection import train_test_split

stores_total_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_train, stores_val = train_test_split(stores_total_train, test_size=0.2, random_state=0)

In [2]:
from utils import create_geographical_columns, split_plaace_cat
revenue_df = split_plaace_cat(create_geographical_columns(stores_train))
revenue_df_total = split_plaace_cat(create_geographical_columns(stores_total_train))
result_df_val = split_plaace_cat(create_geographical_columns(stores_val))
result_df_test = split_plaace_cat(create_geographical_columns(stores_test))
result_df_test_total = split_plaace_cat(create_geographical_columns(stores_test))

In [60]:
from constants import *
result_df = revenue_df.copy()
revenue_df_dict : dict[str, pd.DataFrame]
revenue_df_dict_total : dict[str, pd.DataFrame]
revenue_df_dict = {}
revenue_df_dict_total = {}
for level in levels:
    grouped_df = revenue_df.groupby([level, 'plaace_cat_1']).mean()['revenue'].to_frame()
    grouped_df_total = revenue_df_total.groupby([level, 'plaace_cat_1']).mean()['revenue'].to_frame()
    grouped_df.rename(columns={'revenue':f'{level}.mean_revenue'}, inplace=True)
    grouped_df_total.rename(columns={'revenue':f'{level}.mean_revenue'}, inplace=True)
    revenue_df_dict[level] = grouped_df
    revenue_df_dict_total[level] = grouped_df_total
    result_df = pd.merge(result_df, grouped_df, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')
    result_df_val = pd.merge(result_df_val, grouped_df, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')
    result_df_test = pd.merge(result_df_test, grouped_df, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')
    result_df_test_total = pd.merge(result_df_test_total, grouped_df_total, right_index=True, left_on=[level, 'plaace_cat_1'], how='left')

KeyError: 'grunnkrets_id'

In [4]:
result_df =  result_df[[f'{level}.mean_revenue' for level in levels]+['store_id', 'revenue']]
result_df_val =  result_df_val[[f'{level}.mean_revenue' for level in levels]+['store_id', 'revenue']]
result_df_test =  result_df_test[[f'{level}.mean_revenue' for level in levels]+['store_id']]
result_df_test_total =  result_df_test_total[[f'{level}.mean_revenue' for level in levels]+['store_id']]

In [5]:
result_df_val.shape

(2572, 6)

In [6]:
result_df_val.isna().sum()

grunnkrets_id.mean_revenue    818
delomrade.mean_revenue        227
kommune.mean_revenue           54
fylke.mean_revenue              0
store_id                        0
revenue                         0
dtype: int64

In [7]:
result_df_test.isna().sum()

grunnkrets_id.mean_revenue    2864
delomrade.mean_revenue         833
kommune.mean_revenue           202
fylke.mean_revenue               0
store_id                         0
dtype: int64

In [8]:
result_df_test_total.isna().sum()

grunnkrets_id.mean_revenue    2541
delomrade.mean_revenue         690
kommune.mean_revenue           159
fylke.mean_revenue               0
store_id                         0
dtype: int64

In [9]:
result_df.corr()

,grunnkrets_id.mean_revenue,delomrade.mean_revenue,kommune.mean_revenue,fylke.mean_revenue,revenue
grunnkrets_id.mean_revenue,1.000000,0.727092,0.489814,0.383120,0.727744
delomrade.mean_revenue,0.727092,1.000000,0.673662,0.526921,0.529137
kommune.mean_revenue,0.489814,0.673662,1.000000,0.782175,0.356459
fylke.mean_revenue,0.383120,0.526921,0.782175,1.000000,0.278813
revenue,0.727744,0.529137,0.356459,0.278813,1.000000


In [10]:
result_df_val.corr()

,grunnkrets_id.mean_revenue,delomrade.mean_revenue,kommune.mean_revenue,fylke.mean_revenue,revenue
grunnkrets_id.mean_revenue,1.000000,0.704734,0.490257,0.377568,0.126271
delomrade.mean_revenue,0.704734,1.000000,0.598586,0.476958,0.111950
kommune.mean_revenue,0.490257,0.598586,1.000000,0.811830,0.190145
fylke.mean_revenue,0.377568,0.476958,0.811830,1.000000,0.257962
revenue,0.126271,0.111950,0.190145,0.257962,1.000000


In [11]:
revenue_df_dict

{'grunnkrets_id':                             grunnkrets_id.mean_revenue
 grunnkrets_id plaace_cat_1                            
 1010105       1                               5.160000
               2                               7.286429
               3                               4.202333
 1010107       3                               1.628000
 1010204       1                               4.304000
 ...                                                ...
 20300303      2                               5.268667
               3                               3.322000
 20300309      3                               2.948000
 20300310      1                               8.584000
               2                              31.833000
 
 [5085 rows x 1 columns],
 'delomrade':                         delomrade.mean_revenue
 delomrade plaace_cat_1                        
 10101     1                           5.160000
           2                           7.286429
           3          

In [12]:
for level, df in revenue_df_dict.items():
    df.to_csv(f'temp_data/{level}_mean_revenue.csv')

In [13]:
for level, df in revenue_df_dict_total.items():
    df.to_csv(f'temp_data/{level}_mean_revenue_total.csv')

In [92]:
from avg_revenue import add_avg_revenue, create_avg_revenue_csvs
from constants import *

create_avg_revenue_csvs()
result = add_avg_revenue(stores_val, total=False)

In [93]:
result.isna().sum()

store_id                                      0
year                                          0
store_name                                    0
plaace_hierarchy_id                           0
sales_channel_name                            0
grunnkrets_id                                 0
address                                     349
lat                                           0
lon                                           0
chain_name                                 1815
mall_name                                  2081
revenue                                       0
fylke                                         0
kommune                                       0
delomrade                                     0
plaace_cat_1                                  0
plaace_cat_2                                  0
plaace_cat_3                                  0
plaace_cat_4                                  0
country                                       0
plaace_cat_0                            

In [95]:
result[['revenue']+ [f'{level}.{cat}_mean_revenue' for level in levels_ext for cat in plaace_cols_ext]].corr()

,revenue,grunnkrets_id.plaace_cat_0_mean_revenue,grunnkrets_id.plaace_cat_1_mean_revenue,grunnkrets_id.plaace_cat_2_mean_revenue,grunnkrets_id.plaace_cat_3_mean_revenue,grunnkrets_id.plaace_cat_4_mean_revenue,delomrade.plaace_cat_0_mean_revenue,delomrade.plaace_cat_1_mean_revenue,delomrade.plaace_cat_2_mean_revenue,delomrade.plaace_cat_3_mean_revenue,...,fylke.plaace_cat_0_mean_revenue,fylke.plaace_cat_1_mean_revenue,fylke.plaace_cat_2_mean_revenue,fylke.plaace_cat_3_mean_revenue,fylke.plaace_cat_4_mean_revenue,country.plaace_cat_0_mean_revenue,country.plaace_cat_1_mean_revenue,country.plaace_cat_2_mean_revenue,country.plaace_cat_3_mean_revenue,country.plaace_cat_4_mean_revenue
revenue,1.000000,0.015376,0.116893,0.267097,0.375266,0.479124,-0.006961,0.122471,0.280674,0.389668,...,-0.010137,0.257962,0.425386,0.512912,0.546424,NaN,0.264806,0.432074,0.530743,0.563171
grunnkrets_id.plaace_cat_0_mean_revenue,0.015376,1.000000,0.574380,0.228491,0.178618,0.033903,0.762634,0.444115,0.199645,0.175216,...,0.087105,0.002270,0.005076,0.032247,-0.001630,NaN,-0.017275,-0.004867,0.008361,-0.016186
grunnkrets_id.plaace_cat_1_mean_revenue,0.116893,0.574380,1.000000,0.444008,0.347106,0.207799,0.556924,0.843772,0.411273,0.344253,...,0.081609,0.403948,0.251207,0.211023,0.167237,NaN,0.379920,0.242696,0.194844,0.159819
grunnkrets_id.plaace_cat_2_mean_revenue,0.267097,0.228491,0.444008,1.000000,0.724183,0.582250,0.269495,0.479439,0.951240,0.710213,...,0.042485,0.370279,0.583031,0.473400,0.414724,NaN,0.357041,0.561709,0.444224,0.394531
grunnkrets_id.plaace_cat_3_mean_revenue,0.375266,0.178618,0.347106,0.724183,1.000000,0.847416,0.216582,0.385132,0.710104,0.968085,...,0.036642,0.312191,0.526520,0.727447,0.656438,NaN,0.302573,0.504794,0.669487,0.607575
grunnkrets_id.plaace_cat_4_mean_revenue,0.479124,0.033903,0.207799,0.582250,0.847416,1.000000,0.034651,0.238512,0.587479,0.856442,...,0.011870,0.318657,0.541065,0.741229,0.780326,NaN,0.315890,0.524560,0.694429,0.733269
delomrade.plaace_cat_0_mean_revenue,-0.006961,0.762634,0.556924,0.269495,0.216582,0.034651,1.000000,0.632255,0.267384,0.220713,...,0.132893,0.028336,0.009861,0.028557,-0.016639,NaN,0.001370,-0.000167,0.004386,-0.028590
delomrade.plaace_cat_1_mean_revenue,0.122471,0.444115,0.843772,0.479439,0.385132,0.238512,0.632255,1.000000,0.495325,0.398127,...,0.086901,0.493736,0.292735,0.232415,0.184688,NaN,0.472203,0.285843,0.215906,0.178459
delomrade.plaace_cat_2_mean_revenue,0.280674,0.199645,0.411273,0.951240,0.710104,0.587479,0.267384,0.495325,1.000000,0.736877,...,0.046946,0.392091,0.628530,0.494350,0.431427,NaN,0.379167,0.608055,0.470587,0.416624
delomrade.plaace_cat_3_mean_revenue,0.389668,0.175216,0.344253,0.710213,0.968085,0.856442,0.220713,0.398127,0.736877,1.000000,...,0.037851,0.326103,0.548354,0.753476,0.685045,NaN,0.317980,0.529383,0.699215,0.639610
